<a href="https://colab.research.google.com/github/paul66853/ML/blob/main/PM25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import sys

In [64]:
import pandas as pd

In [65]:
import numpy as np

In [66]:
from google.colab import drive

In [67]:
!gdown --id '1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm' --output data.zip
!unzip data.zip
data = pd.read_csv('./train.csv',encoding='big5')

Downloading...
From: https://drive.google.com/uc?id=1wNKAxQ29G15kgpBy_asjTcZRRgmsCZRm
To: /content/data.zip
100% 177k/177k [00:00<00:00, 54.6MB/s]
Archive:  data.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ㄗ
error:  invalid response [ㄗ]
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [68]:
data=data.iloc[:,3:]#iloc透過行索引的 Index 的具體值來取行資料（如data.iloc[:，:]data[data == 'xxx'] = 0）
print(data)
print(data.shape)
data[data=='NR']=0
print(type(data))
raw_data=data.to_numpy()

         0     1     2     3     4     5  ...    18    19    20    21    22    23
0       14    14    14    13    12    12  ...    17    16    15    15    15    15
1      1.8   1.8   1.8   1.8   1.8   1.8  ...   1.8   1.8   1.8   1.8   1.8   1.8
2     0.51  0.41  0.39  0.37  0.35   0.3  ...  0.56  0.45  0.38  0.35  0.36  0.32
3      0.2  0.15  0.13  0.12  0.11  0.06  ...  0.18  0.12   0.1  0.09   0.1  0.08
4      0.9   0.6   0.5   1.7   1.8   1.5  ...   2.1   1.9   1.5   1.6   1.8   1.5
...    ...   ...   ...   ...   ...   ...  ...   ...   ...   ...   ...   ...   ...
4315   1.8   1.8   1.8   1.8   1.8   1.7  ...     2   1.9   1.9   1.9     2     2
4316    46    13    61    44    55    68  ...   100   109   108   114   108   109
4317    36    55    72   327    74    52  ...   121    97   107   118   100   105
4318   1.9   2.4   1.9   2.8   2.3   1.9  ...   1.5     1   1.7   1.5     2     2
4319   0.7   0.8   1.8     1   1.9   1.7  ...   1.1   1.4   1.3   1.6   1.8     2

[4320 rows x 24

In [69]:
month_data={}
for month in range(12):
  sample = np.empty([18,480])#新建NP數組內含18種數據，20天，每天24小時，總共480個小時
  for day in range(20):
    sample[:,day*24:(day+1)*24]=raw_data[18*(20*month+day):18*(20*month+day+1),:]#從原先的轉換成12*18*(20*24)
  month_data[month]=sample

In [70]:
x = np.empty([12 * 471,18 * 9],dtype = float)
y = np.empty([12 * 471,1],dtype = float)
for month in range(12):
  for day in range(20):
    for hour in range(24):
      if day == 19 and hour > 14:
        continue
      x[month * 471+day * 24 + hour,:] = month_data[month][:,day*24+hour:day*24+hour+9].reshape(1,-1)
      y[month * 471+day * 24 + hour,0] = month_data[month][9,day * 24 + hour + 9]
print(x)
print(x.shape)
print(y)
print(y.shape)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
(5652, 162)
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]
(5652, 1)


In [71]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
x

array([[-1.35825331, -1.35883937, -1.359222  , ...,  0.26650729,
         0.2656797 , -1.14082131],
       [-1.35825331, -1.35883937, -1.51819928, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [-1.35825331, -1.51789368, -1.67717656, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [-0.88092053, -0.72262212, -0.56433559, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [-0.7218096 , -0.56356781, -0.72331287, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [-0.56269867, -0.72262212, -0.88229015, ..., -0.38950555,
        -0.10906991,  0.07797893]])

In [72]:
import math
x_train_set = x[: math.floor(len(x)*0.8), :] #前80%用於訓練
y_train_set = y[: math.floor(len(y)*0.8), :]  
x_validation = x[math.floor(len(x) *0.8): , :]#20%用於驗證 #floor返回不大於x的最大整數math.floor(100.12):100.0
y_validation = y[math.floor(len(y) *0.8): , :]  
print(x_train_set)
print(y_train_set)
print(x_validation)
print(y_validation)

[[-1.35825331 -1.35883937 -1.359222   ...  0.26650729  0.2656797
  -1.14082131]
 [-1.35825331 -1.35883937 -1.51819928 ...  0.26650729 -1.13963133
  -1.32832904]
 [-1.35825331 -1.51789368 -1.67717656 ... -1.13923451 -1.32700613
  -0.85955971]
 ...
 [ 0.86929969  0.70886668  0.38952809 ...  1.39110073  0.2656797
  -0.39079039]
 [ 0.71018876  0.39075806  0.07157353 ...  0.26650729 -0.39013211
  -0.39079039]
 [ 0.3919669   0.07264944  0.07157353 ... -0.38950555 -0.39013211
  -0.85955971]]
[[30.]
 [41.]
 [44.]
 ...
 [ 7.]
 [ 5.]
 [14.]]
[[ 0.07374504  0.07264944  0.07157353 ... -0.38950555 -0.85856912
  -0.57829812]
 [ 0.07374504  0.07264944  0.23055081 ... -0.85808615 -0.57750692
   0.54674825]
 [ 0.07374504  0.23170375  0.23055081 ... -0.57693779  0.54674191
  -0.1095288 ]
 ...
 [-0.88092053 -0.72262212 -0.56433559 ... -0.57693779 -0.29644471
  -0.39079039]
 [-0.7218096  -0.56356781 -0.72331287 ... -0.29578943 -0.39013211
  -0.1095288 ]
 [-0.56269867 -0.72262212 -0.88229015 ... -0.3895055

In [73]:
dim = 18*9+1  #維度，因為常數項，需多加一欄
w = np.zeros([dim,1]) #生成0的數據
x = np.concatenate((np.ones([12*471,1]),x),axis=1).astype(float)  #拼接1和X
learning_rate = 100 #學習率
iter_time = 1000  #學習次數
adagrad = np.zeros([dim, 1])  #生成0的數據
eps = 0.0000000001
for t in range(iter_time):
    loss = np.sqrt(np.sum(np.power(np.dot(x, w) - y, 2))/471/12)#rmse
    if(t%100==0):#每訓練100次印出一次LOSS值
        print(str(t) + ":" + str(loss))
    gradient = 2 * np.dot(x.transpose(), np.dot(x, w) - y) #dim*1
    adagrad += gradient ** 2
    w = w - learning_rate * gradient / (np.sqrt(adagrad) + eps) 
np.save('weight.npy', w)
w

0:27.071214829194115
100:33.78905859777102
200:19.91375129819506
300:13.531068193688462
400:10.645466158445366
500:9.277353455474486
600:8.518042045956047
700:8.014061987588049
800:7.636756824775373
900:7.33656374037085


array([[ 2.13740269e+01],
       [ 3.58888909e+00],
       [ 4.56386323e+00],
       [ 2.16307023e+00],
       [-6.58545223e+00],
       [-3.38885580e+01],
       [ 3.22235518e+01],
       [ 3.49340354e+00],
       [-4.60308671e+00],
       [-1.02374754e+00],
       [-3.96791501e-01],
       [-1.06908800e-01],
       [ 2.22488184e-01],
       [ 8.99634117e-02],
       [ 1.31243105e-01],
       [ 2.15894989e-02],
       [-1.52867263e-01],
       [ 4.54087776e-02],
       [ 5.20999235e-01],
       [ 1.60824213e-01],
       [-3.17709451e-02],
       [ 1.28529025e-02],
       [-1.76839437e-01],
       [ 1.71241371e-01],
       [-1.31190032e-01],
       [-3.51614451e-02],
       [ 1.00826192e-01],
       [ 3.45018257e-01],
       [ 4.00130315e-02],
       [ 2.54331382e-02],
       [-5.04425219e-01],
       [ 3.71483018e-01],
       [ 8.46357671e-01],
       [-8.11920428e-01],
       [-8.00217575e-02],
       [ 1.52737711e-01],
       [ 2.64915130e-01],
       [-5.19860416e-02],
       [-2.5

In [74]:
testdata = pd.read_csv('./test.csv', header = None, encoding = 'big5')
test_data = testdata.iloc[:, 2:]  #去除文件第一列的ID和名稱，只留下數據
test_data[test_data == 'NR'] = 0  #跟前面一樣，將NR改成0
test_data = test_data.to_numpy()
test_x = np.empty([240, 18*9], dtype = float)#18*9是特徵的數量，240組，每組18*9個
for i in range(240):
    test_x[i, :] = test_data[18 * i: 18* (i + 1), :].reshape(1, -1)#每隔18個數據為一行
for i in range(len(test_x)):      #行
    for j in range(len(test_x[0])): #列
        if std_x[j] != 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]
test_x = np.concatenate((np.ones([240, 1]), test_x), axis = 1).astype(float)
test_x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3093: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([[ 1.        , -0.24447681, -0.24545919, ..., -0.67065391,
        -1.04594393,  0.07797893],
       [ 1.        , -1.35825331, -1.51789368, ...,  0.17279117,
        -0.10906991, -0.48454426],
       [ 1.        ,  1.5057434 ,  1.34508393, ..., -1.32666675,
        -1.04594393, -0.57829812],
       ...,
       [ 1.        ,  0.3919669 ,  0.54981237, ...,  0.26650729,
        -0.20275731,  1.20302531],
       [ 1.        , -1.8355861 , -1.8360023 , ..., -1.04551839,
        -1.13963133, -1.14082131],
       [ 1.        , -1.35825331, -1.35883937, ...,  2.98427476,
         3.26367657,  1.76554849]])

In [75]:
w = np.load('weight.npy')  # 讀取剛剛的文檔
ans_y = np.dot(test_x, w)  # 預測
ans_y

array([[ 5.17496040e+00],
       [ 1.83062143e+01],
       [ 2.04912181e+01],
       [ 1.15239429e+01],
       [ 2.66160568e+01],
       [ 2.05313481e+01],
       [ 2.19065510e+01],
       [ 3.17364687e+01],
       [ 1.33916741e+01],
       [ 6.44564665e+01],
       [ 2.02645688e+01],
       [ 1.53585761e+01],
       [ 6.85894728e+01],
       [ 4.84281137e+01],
       [ 1.87023338e+01],
       [ 1.01885957e+01],
       [ 3.07403629e+01],
       [ 7.11322178e+01],
       [-4.13051739e+00],
       [ 1.82356940e+01],
       [ 3.85789223e+01],
       [ 7.13115197e+01],
       [ 7.41034816e+00],
       [ 1.87179553e+01],
       [ 1.49372503e+01],
       [ 3.67197367e+01],
       [ 1.79616970e+01],
       [ 7.57894629e+01],
       [ 1.23093102e+01],
       [ 5.62953517e+01],
       [ 2.51131609e+01],
       [ 4.61024867e+00],
       [ 2.48377055e+00],
       [ 2.47594223e+01],
       [ 3.04802805e+01],
       [ 3.84639307e+01],
       [ 4.42023106e+01],
       [ 3.00868360e+01],
       [ 4.0

In [76]:
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 5.174960398985339]
['id_1', 18.306214253527465]
['id_2', 20.491218094181523]
['id_3', 11.523942869806074]
['id_4', 26.616056752306218]
['id_5', 20.53134808176113]
['id_6', 21.906551018797497]
['id_7', 31.73646874706836]
['id_8', 13.391674055112954]
['id_9', 64.45646650291695]
['id_10', 20.26456883615848]
['id_11', 15.358576077360935]
['id_12', 68.58947276926689]
['id_13', 48.428113747456926]
['id_14', 18.70233382419282]
['id_15', 10.188595737466832]
['id_16', 30.740362859821374]
['id_17', 71.13221776355063]
['id_18', -4.130517391262495]
['id_19', 18.235694016428123]
['id_20', 38.57892227500801]
['id_21', 71.31151972531325]
['id_22', 7.41034816263461]
['id_23', 18.71795533032117]
['id_24', 14.937250260084268]
['id_25', 36.71973669470527]
['id_26', 17.961697005661964]
['id_27', 75.78946287210414]
['id_28', 12.309310248614722]
['id_29', 56.29535173964829]
['id_30', 25.113160865661353]
['id_31', 4.61024867409447]
['id_32', 2.4837705545160413]
['id_33', 24.759422261